In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

url = 'https://en.wikipedia.org/wiki/List_of_best-selling_albums_in_the_United_States'
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

column_titles = soup.find_all('th')
column_titles_table = [title.text.strip() for title in column_titles]

# initial data cleaning
index = 5
del column_titles_table[index+1:]
column_titles_table[4] = "Sales"
column_titles_table[5] = "Certification"

tables = soup.find_all('table', class_='wikitable sortable')

df = pd.DataFrame(columns = column_titles_table)

for table in tables:
    rows = table.find_all('tr')
    for row in rows[1:]:
        cells = row.find_all('td')
        if len(cells) == len(df.columns):
            row_data = [cell.text.strip() for cell in cells]
            df.loc[len(df)] = row_data

# Pandas data cleaning

# Drop rows with any missing values
df = df.dropna()

# Remove special characters from all columns (only non-numeric columns)
for column in df.columns:
    if df[column].dtype == 'object':  # Check if column is string
        df[column] = df[column].apply(lambda x: ''.join(e for e in str(x) if e.isalnum() or e.isspace()))  # Keep only letters, numbers, spaces

df.to_csv(r'/Users/brendan/Downloads/Output Folder/Albums.csv', index = False)
